In [ ]:
from scipy.stats import norm
from numpy import sqrt, log, exp

# The Black-Scholes Option Pricing Formula

In its basic form, the Black-Scholes formula for pricing European-style options is given by:

$$C = S \cdot N(d_1) - K \cdot e^{-rT} \cdot N(d_2)$$
$$P = K \cdot e^{-rT} \cdot N(-d_2) - S \cdot N(-d_1)$$

where:
- $C$ is the price of the call option
- $P$ is the price of the put option
- $S$ is the current price of the underlying asset
- $K$ is the strike price of the option
- $r$ is the risk-free interest rate
- $T$ is the time to expiration (in years)
- $e$ is the base of the natural logarithm (approximately 2.71828)
- $N(x)$ represents the cumulative standard normal distribution function
- $d1$ and $d2$ are calculated as follows:
  $$d_1 = \frac{\ln(S/K) + (r + \frac{\sigma^2}{2})T}{\sigma \sqrt{T}}$$
  $$d_2 = d_1 - \sigma \sqrt{T}$$
- $\sigma$ is the volatility of the underlying asset

The formula assumes certain assumptions and limitations and is primarily applicable to European-style options on non-dividend-paying stocks.


## Python implementation

In [ ]:
def d1(S, K, T, sigma, r):
    return (log(S/K) + (r+sigma**2/2)*T) / (sigma*sqrt(T))

def d2(S, K, T, sigma, r):
    return d1(S, K, T, sigma, r) - sigma*sqrt(T)

def Black_Scholes_price(S, K, T, c_p, sigma, r):
    if c_p == 'c':
        return S * norm.cdf(d1(S, K, T, sigma, r)) - K * exp(-r*T) * norm.cdf(d2(S, K, T, sigma, r))
    elif c_p == 'p':
        return K * exp(-r*T) * norm.cdf(-d2(S, K, T, sigma, r)) - S * norm.cdf(-d1(S, K, T, sigma, r))

# The Option Greeks

The option Greeks are measures that quantify the sensitivity of option prices to changes in various factors. They help traders and investors understand and manage the risks associated with options. The main option Greeks are as follows:

### Delta:
Delta measures the rate of change of the option price relative to changes in the price of the underlying asset.

For a call option:
$$\Delta_{\text{call}} = N(d_1)$$
For a put option:
$$\Delta_{\text{put}} = N(d_1) - 1 = -N(-d_1)$$

### Gamma:
Gamma represents the rate of change of the option's delta with respect to changes in the price of the underlying asset.
$$\Gamma = \frac{N'(d_1)}{S \cdot \sigma \cdot \sqrt{T}}$$

### Theta:
Theta measures the rate of change of the option price over time, often referred to as time decay.

For a call option:
$$\Theta_{\text{call}} = -\frac{S \cdot N'(d_1) \cdot \sigma}{2\sqrt{T}} - r \cdot K \cdot e^{-rT} \cdot N(d_2)$$
For a put option:
$$\Theta_{\text{put}} = -\frac{S \cdot N'(d_1) \cdot \sigma}{2\sqrt{T}} + r \cdot K \cdot e^{-rT} \cdot N(-d_2)$$

### Vega:
Vega quantifies the sensitivity of the option price to changes in the implied volatility of the underlying asset.
$$Vega = S \cdot \sqrt{T} \cdot N'(d_1)$$

### Rho:
Rho indicates the sensitivity of the option price to changes in the risk-free interest rate.

For a call option:
$$\rho_{\text{call}} = K \cdot T \cdot e^{-rT} \cdot N(d_2)$$
For a put option:
$$\rho_{\text{put}} = -K \cdot T \cdot e^{-rT} \cdot N(-d_2)$$

where:
- $N(x)$ represents the cumulative standard normal distribution function
- $N'(x)$ represents the probability density function of the standard normal distribution
- $S$ is the current price of the underlying asset
- $K$ is the strike price of the option
- $r$ is the risk-free interest rate
- $\sigma$ is the implied volatility of the underlying asset
- $T$ is the time to expiration (in years)
- $d_1$ and $d_2$ are calculated as follows:
   $$d_1 = \frac{\ln\left(\frac{S}{K}\right) + \left(r + \frac{\sigma^2}{2}\right)T}{\sigma \sqrt{T}}$$
   $$d_2 = d_1 - \sigma \sqrt{T}$$

Note that these formulas are simplified and assume certain assumptions and limitations. They are primarily applicable to European-style options on non-dividend-paying stocks.

Each Greek provides unique insights into the behavior of options and can be used to manage and adjust options positions based on changing market conditions.

## Python implementation

In [ ]:
def delta(S, K, T, c_p, sigma, r):
    if c_p == 'c':
        return norm.cdf(d1(S, K, T, sigma, r))
    elif c_p == 'p':
        return -norm.cdf(-d1(S, K, T, sigma, r))

def gamma(S, K, T, c_p, sigma, r):
    return norm.pdf(d1(S, K, T, sigma, r)) / (S*sigma*sqrt(T))

def theta(S, K, T, c_p, sigma, r):
    if c_p == 'c':
        return (-(S*norm.pdf(d1(S, K, T, sigma, r))*sigma)/(2*sqrt(T)) - r*K*exp(-r*T)*norm.cdf(d2(S, K, T, sigma, r))) / 365
    elif c_p == 'p':
        return (-(S*norm.pdf(d1(S, K, T, sigma, r))*sigma)/(2*sqrt(T)) + r*K*exp(-r*T)*norm.cdf(-d2(S, K, T, sigma, r))) / 365

def vega(S, K, T, c_p, sigma, r):
    return 0.01*(S*sqrt(T)*norm.pdf(d1(S, K, T, sigma, r)))

def rho(S, K, T, c_p, sigma, r):
    if c_p == 'c':
        return 0.01*(K*T*exp(-r*T)*norm.cdf(d2(S, K, T, sigma, r)))
    elif c_p == 'p':
        return 0.01*(-K*T*exp(-r*T)*norm.cdf(-d2(S, K, T, sigma, r)))

# Implied Volatility

Implied volatility is the estimated volatility of the underlying asset that makes the calculated option price equal to the market price. It is a measure of the market's expectation of future volatility of the underlying asset. The Black-Scholes formula can be used to estimate implied volatility.

To calculate implied volatility, we can use the following steps:

1. Define the Black-Scholes formula for an option price:
$$C = S \cdot N(d_1) - K \cdot e^{-rT} \cdot N(d_2)$$
where $C$ is the call option price, $S$ is the current price of the underlying asset, $K$ is the strike price of the option, $r$ is the risk-free interest rate, $T$ is the time to expiration (in years), $N(x)$ is the cumulative standard normal distribution function, and $d_1$ and $d_2$ are calculated as follows:
$$d_1 = \frac{\ln\left(\frac{S}{K}\right) + \left(r + \frac{\sigma^2}{2}\right)T}{\sigma \sqrt{T}}$$
$$d_2 = d_1 - \sigma \sqrt{T}$$
Here, $\sigma$ represents the implied volatility.


2. To calculate implied volatility, you need to solve the above equations for the implied volatility ($\sigma$). This involves iteratively finding the value of $\sigma$ that makes the calculated option price ($C$ or $P$) equal to the market price.

Note that the Black-Scholes formula assumes certain assumptions and limitations, such as constant volatility, no dividends, and efficient markets.

Keep in mind that implied volatility is just an estimation and may vary depending on market conditions and the assumptions made in the option pricing model.



### Implied Volatility – Brute Force Method

In [ ]:
def iv_brute_force(S, K, T, c_p, r, market_price):
    MAX_TRY = 10000 # max number of calculations
    ONE_CENT = 0.01 # acceptable difference between the calculated and market prices
    STEP = 0.0001   # step variable to adjust the sigma values
    sigma = 0.1     # initial guess of sigma value
    for i in range(MAX_TRY):
        bs_price = Black_Scholes_price(S, K, T, c_p, sigma, r)
        diff = market_price - bs_price
        if diff > ONE_CENT:
            sigma += STEP
        elif diff < -ONE_CENT:
            sigma -= STEP
        else:
            return sigma
    return sigma

### Implied Volatility – Newton Raphson Method

In [ ]:
def iv_newton(S, K, T, c_p, r, market_price):
    MAX_TRY = 1000  # max number of calculations
    ONE_CENT = 0.01 # acceptable difference between the calculated and market prices
    sigma = 0.1     # initial guess of sigma value
    for i in range(MAX_TRY):
        bs_price = Black_Scholes_price(S, K, T, c_p, sigma, r)
        diff = market_price - bs_price
        if abs(diff) < ONE_CENT:
            return sigma
        sigma += diff / (vega(S, K, T, c_p, sigma, r)*100)
    return sigma

### Implied Volatility - optimize.brentq

In [ ]:
'''
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.brentq.html
'''
from scipy.optimize import brentq
def iv_brentq(S, K, T, c_p, r, market_price):

    def prices_diff(sigma):
        return Black_Scholes_price(S, K, T, c_p, sigma, r) - market_price

    return brentq(prices_diff, 0.0001, 10, maxiter=1000)

### Implied Volatility - optimize.fsolve

In [ ]:
'''
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fsolve.html
'''
from scipy.optimize import fsolve
def iv_fsolve(S, K, T, c_p, r, market_price):

    def prices_diff(sigma):
        return Black_Scholes_price(S, K, T, c_p, sigma, r) - market_price

    return fsolve(prices_diff, 0.1)[0]

# Run tests

In [ ]:
# input parameters
S = 356.61  # current price of the underlying asset
c_p = 'p'   # option type: 'c' - call, 'p' - put
K = 324     # strike price of the option
T = 65/365  # time to expiration in 'days/365'
r = 0       # risk-free interest rate

sigma = 0.3279      # volatility of the asset
market_price = 6.79 # option market price for IV calculation

# run tests
print ('Black Scholes Price:', Black_Scholes_price(S, K, T, c_p, sigma, r))
print ('Delta:', delta(S, K, T, c_p, sigma, r))
print ('Gamma:', gamma(S, K, T, c_p, sigma, r))
print ('Vega: ', vega(S, K, T, c_p, sigma, r))
print ('Theta:', theta(S, K, T, c_p, sigma, r))
print ('Rho:  ', rho(S, K, T, c_p, sigma, r))
print ('IV Brute Force:    ', iv_brute_force(S, K, T, c_p, r, market_price))
print ('IV Newton:         ', iv_newton(S, K, T, c_p, r, market_price))
print ('IV optimize.brentq:', iv_brentq(S, K, T, c_p, r, market_price))
print ('IV optimize.fsolve:', iv_fsolve(S, K, T, c_p, r, market_price))